# 2.6 정수 인코딩(Integer Encoding)

컴퓨터는 텍스트보다는 숫자를 더 잘 처리 할 수 있습니다. 

이를 위해 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법들이 있습니다. 
그리고 그러한 기법들을 본격적으로 적용시키기 위한 첫 단계로 각 단어를 고유한 정수에 맵핑(mapping)시키는 전처리 작업이 필요할 때가 있습니다.

예를 들어 
- 갖고 있는 텍스트에 단어가 5,000개가 있다면, 
- 5,000개의 단어들 각각에 1번부터 5,000번까지 단어와 맵핑되는 고유한 정수, 다른 표현으로는 인덱스를 부여합니다. 
- 가령, book은 150번, dog는 171번, love는 192번, books는 212번과 같이 숫자가 부여됩니다. 
- 인덱스를 부여하는 방법은 여러 가지가 있을 수 있는데 랜덤으로 부여하기도 하지만, 
- 보통은 전처리 또는 **빈도수가 높은 단어**들만 사용하기 위해서 **단어에 대한 빈도수를 기준으로 정렬**한 뒤에 부여합니다.

### 1. 정수 인코딩(Integer Encoding)
왜 이러한 작업이 필요한 지에 대해서는 뒤에서 원-핫 인코딩, 워드 임베딩 챕터 등에서 알아보기로 하고 여기서는 어떤 과정으로 단어에 정수 인덱스를 부여하는지에 대해서만 정리

단어에 정수를 부여하는 방법 중 하나로 
1. 단어를 빈도수 순으로 정렬한 단어 집합(vocabulary)을 만들고, 
2. 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여 

##### 1) dictionary 사용하기

In [67]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

text = "A barber is a person. a barber is good person. a barber is huge person. he Knew A Secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [68]:
# Tokenize sentence from corpus
tkz_sentences = sent_tokenize(text)
print(tkz_sentences)

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he Knew A Secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [70]:
# cleaning and Tokenize 
vocab = {}  # Dictionary to store the frequency of the word's appearance
rfd_sentences = [] #  List to store tokenized text
stop_words = set(stopwords.words('english'))

# Separate text into sentence units
for i in tkz_sentences:
    sentence = word_tokenize(i)
    
    result =[]
    
    # Separate setenences into each word
    for word in sentence:
        word = word.lower()
        
        # if the word is not in stopwords
        if word not in stop_words:
            
            # if the word is more than 3 in length
            if len(word)>2:
                result.append(word)
                
                # if the word is not in vocab
                if word not in vocab:
                    vocab[word] = 0
                vocab[word] +=1
    rfd_sentences.append(result)
    print(">>>",sentence, "==>",result)
print("=======================") 
print(vocab)
print("=======================") 
print(rfd_sentences)  

>>> ['A', 'barber', 'is', 'a', 'person', '.'] ==> ['barber', 'person']
>>> ['a', 'barber', 'is', 'good', 'person', '.'] ==> ['barber', 'good', 'person']
>>> ['a', 'barber', 'is', 'huge', 'person', '.'] ==> ['barber', 'huge', 'person']
>>> ['he', 'Knew', 'A', 'Secret', '!'] ==> ['knew', 'secret']
>>> ['The', 'Secret', 'He', 'Kept', 'is', 'huge', 'secret', '.'] ==> ['secret', 'kept', 'huge', 'secret']
>>> ['Huge', 'secret', '.'] ==> ['huge', 'secret']
>>> ['His', 'barber', 'kept', 'his', 'word', '.'] ==> ['barber', 'kept', 'word']
>>> ['a', 'barber', 'kept', 'his', 'word', '.'] ==> ['barber', 'kept', 'word']
>>> ['His', 'barber', 'kept', 'his', 'secret', '.'] ==> ['barber', 'kept', 'secret']
>>> ['But', 'keeping', 'and', 'keeping', 'such', 'a', 'huge', 'secret', 'to', 'himself', 'was', 'driving', 'the', 'barber', 'crazy', '.'] ==> ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy']
>>> ['the', 'barber', 'went', 'up', 'a', 'huge', 'mountain', '.'] ==> ['barber', 'went',

In [19]:
print(vocab["barber"])   # Output the frequency of the appearance of "barber" words

8


In [21]:
# Arrange in order of appearance frequency
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)
print(vocab_sorted)

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


In [72]:
# Set lower index for words with higher frequency
idx_vocab = {}
i = 0

for ( word, frequency ) in vocab_sorted:
    # Exclude less frequently used words
    if frequency > 1:
        i = i + 1
        idx_vocab[word] = i
print(idx_vocab)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


In [78]:
# Use only the top five words
vocab_size = 5

words_frequency = [ w for w, c in idx_vocab.items() if c >= vocab_size+1]

for w in words_frequency:
    del idx_vocab[w]
print(idx_vocab)    

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


이제 word_to_index에는 빈도수가 높은 상위 5개의 단어만 저장되었습니다. 이제 word_to_index를 사용하여 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾸는 작업을 하겠습니다.

예를 들어 sentences에서 첫번째 문장은 ['barber', 'person']이었는데, 이 문장에 대해서는 [1, 5]로 인코딩합니다. 그런데 두번째 문장인 ['barber', 'good', 'person']에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어가 있습니다.

이처럼 훈련 데이터 또는 테스트 데이터에 대해서 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary(단어 집합에 없는 단어)의 약자로 'OOV'라고 합니다. word_to_index에 'OOV'란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 'OOV'의 인덱스로 인코딩하겠습니다.

In [79]:
idx_vocab['OOV'] = len(idx_vocab)+1

이제 word_to_index를 사용하여 sentences의 모든 단어들을 맵핑되는 정수로 인코딩하겠습니다.

In [82]:
encd_sentences = []

# sentences: List of tokenized text
for s in rfd_sentences:
    temp =[]
    
    # Separate setenence into each word
    for w in s:
        try:
            temp.append(idx_vocab[w])
        except KeyError:
            temp.append(idx_vocab['OOV'])
    encd_sentences.append(temp)
print(encd_sentences)    

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


### 2) Counter 사용하기

In [29]:
from collections import Counter

In [83]:
print(rfd_sentences)

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


현재 sentences는 단어 토큰화가 된 결과가 저장되어져 있습니다. 단어 집합(vocabulary)을 만들기 위해서 sentences에서 문장의 경계인 [, ]를 제거하고 단어들을 하나의 리스트로 만들겠습니다.

In [85]:
words = sum(rfd_sentences,[])
print(words)

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


이를 파이썬의 Counter()의 입력으로 사용하면 중복을 제거하고 단어의 빈도수를 기록합니다.

In [94]:
vocab = Counter(words)
print(vocab)

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [95]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [96]:
idx_vocab = {}
i = 0

for ( word, frequency) in vocab:
    i = i+1
    idx_vocab[word]=i
print(idx_vocab)    

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


##### 3) NLTK의 FreqDist 사용하기
NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원합니다. 위에서 사용한 Counter()랑 같은 방법으로 사용할 수 있습니다.

In [36]:
from nltk import FreqDist
import numpy as np

In [90]:
vocab = FreqDist(np.hstack(rfd_sentences))

In [39]:
print(vocab["barber"])

8


In [40]:
vocab_size = 5
vocab = vocab.most_common(vocab_size)
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

앞서 Counter()를 사용했을 때와 결과가 같습니다. 이전 실습들과 마찬가지로 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여합니다. 그런데 이번에는 enumerate()를 사용하여 좀 더 짧은 코드로 인덱스를 부여하겠습니다.

In [97]:
idx_vocab = {word[0]:index+1 for index, word in enumerate(vocab)}
print(idx_vocab)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


**enumerate()**

enumerate()는 순서가 있는 자료형(list, set, tuple, dictionary, string)을 입력으로 받아 인덱스를 순차적으로 함께 리턴한다는 특징이 있습니다. 간단한 예제를 통해 enumerate()를 이해해봅시다.

In [42]:
test = ['a','b', 'c', 'd', 'e']
# 입력의 순서대로 0부터 인덱스를 부여함
for index, value in enumerate(test):
    print("value : {}, index: {}".format(value,index))

value : a, index: 0
value : b, index: 1
value : c, index: 2
value : d, index: 3
value : e, index: 4


### 2. 케라스(Keras)의 텍스트 전처리

케라스(Keras)는 기본적인 전처리를 위한 도구들을 제공합니다. 때로는 정수 인코딩을 위해서 케라스의 전처리 도구인 토크나이저를 사용하기도 하는데, 사용 방법과 그 특징에 대해서 이해해보겠습니다.

In [98]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [99]:
rfd_sentences=[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]

In [100]:
tokenizer = Tokenizer()
# fit_on_texts()안에 코퍼스를 입력으로 하면 
# 빈도수를 기준으로 단어 집합을 생성한다.
tokenizer.fit_on_texts(rfd_sentences)

In [101]:
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


In [50]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [52]:
# texts_to_sequences()는 입력으로 들어온 코퍼스에 대해서 
# 각 단어를 이미 정해진 인덱스로 변환합니다.
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


앞서 빈도수가 가장 높은 단어 n개만을 사용하기 위해서 most_common()을 사용했었습니다. 케라스 토크나이저에서는 tokenizer = Tokenizer(num_words=숫자)와 같은 방법으로 빈도수가 높은 상위 몇 개의 단어만 사용하겠다고 지정할 수 있습니다. 여기서는 1번 단어부터 5번 단어까지만 사용하겠습니다. 상위 5개 단어를 사용한다고 토크나이저를 재정의 해보겠습니다.

In [54]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size + 1)
tokenizer.fit_on_texts(sentences)
print(tokenizer.word_index)

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


상위 5개의 단어만 사용하겠다고 선언하였는데 여전히 13개의 단어가 모두 출력됩니다. word_counts를 확인해보겠습니다.

In [55]:
print(tokenizer.word_counts)

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


word_counts에서도 마찬가지로 13개의 단어가 모두 출력됩니다. 사실 실제 적용은 texts_to_sequences를 사용할 때 적용이 됩니다.

In [56]:
print(tokenizer.texts_to_sequences(sentences))

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


코퍼스에 대해서 각 단어를 이미 정해진 인덱스로 변환하는데, 상위 5개의 단어만을 사용하겠다고 지정하였으므로 1번 단어부터 5번 단어까지만 보존되고 나머지 단어들은 제거된 것을 볼 수 있습니다. 경험상 굳이 필요하다고 생각하지는 않지만, 만약 word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면 아래의 코드도 방법입니다.

In [57]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

In [59]:
vocab_size = 5
words_frequency = [ w for w,c in tokenizer.word_index.items() if c>=vocab_size+1]

for w in words_frequency:
    del tokenizer.word_index[w]
    del tokenizer.word_counts[w]

print(tokenizer.word_index)
print(tokenizer.word_counts)
print(tokenizer.texts_to_sequences(sentences))

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


케라스 토크나이저는 기본적으로 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있습니다. 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용합니다.

In [62]:
vocab_size = 5
tokenizer = Tokenizer(num_words = vocab_size+2, oov_token = 'OOV')
# 빈도수 상위 5개 단어만 사용. 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts(sentences)

만약 oov_token을 사용하기로 했다면 케라스 토크나이저는 기본적으로 'OOV'의 인덱스를 1로 합니다.

In [63]:
print('단어 OOV의 인덱스 : {}'.format(tokenizer.word_index['OOV']))

단어 OOV의 인덱스 : 1


In [64]:
print(tokenizer.texts_to_sequences(sentences))

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


빈도수 상위 5개의 단어는 2 ~ 6까지의 인덱스를 가졌으며, 그 외 단어 집합에 없는 'good'과 같은 단어들은 전부 'OOV'의 인덱스인 1로 인코딩되었습니다.